In [ ]:
# dependencies
import numpy as np
import matplotlib as plt
import pandas as pd
%timeit
import random
import requests  #requests library
#
%run functions.ipynb
from config import *

In [ ]:
from datetime import datetime
from config import *
import numpy as np

city_data = list()

def get_weather_by_city(cities):
    # data hold

    # logging starts.
    print("-----------------------------")
    print("Beginning Data Retrieval     ")
    print("-----------------------------")
    #
    # set counters.
    set_count = 1
    record_count = 0
    #
#     for i, lati, longi in enumerate(coordinates)
    for i, city in enumerate(cities):
        #     group cities by sets 50
        if (i % 50 == 0 and i >= 50):
            set_count += 1
            record_count = 1

        # deal with city with spaces in names
#         search_criterion = "q=" + cities[i].replace(" ", "+")
        search_criterion = "q=" + \
            cities[i].replace(" ", "+")+"&units=imperial"+"&APPID="+weather_api_key
        # create endpoint url for each city
#         city_url = "http://api.openweathermap.org/data/2.5/weather?lat={lati}&lon={longi}&appid={weather_api_key}"
        city_url = "http://api.openweathermap.org/data/2.5/weather?" + search_criterion

        # Log the URL, record, and set numbers and the city.
        # print(city_url)

        print(f"Processing Record {record_count} of Set {set_count} | {city}")
        # increment record count.
        record_count += 1

        # Run an API request for each of the cities.
        try:
            # Parse the JSON and retrieve data.
            city_weather = requests.get(city_url).json()
            # Parse out the needed data.
            city_lat = city_weather["coord"]["lat"]
            city_lng = city_weather["coord"]["lon"]
            city_max_temp = city_weather["main"]["temp_max"]
            city_humidity = city_weather["main"]["humidity"]
            city_clouds = city_weather["clouds"]["all"]
            city_wind = city_weather["wind"]["speed"]
            city_country = city_weather["sys"]["country"]
            city_weather_description = city_weather["weather"]["description"]
            print(city_weather_description)
            # Convert the date to ISO standard.
            city_date = datetime.utcfromtimestamp(
                city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            # Append the city information into city_data list.
            #print(city.title())
            city_data.append({
                "City": city.title(),
                "Lat": city_lat,
                "Lng": city_lng,
                "Max Temp": city_max_temp,
                "Humidity": city_humidity,
                "Cloudiness": city_clouds,
                "Wind Speed": city_wind,
                "Country": city_country,
                "Current Description": city_weather_description,
                "Date": city_date
            })

    # If an error is experienced, skip the city.
        except:
            print("City not found. Skipping...")
            pass

    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")
    return city_data

In [ ]:
# %run functions.ipynb
%run api_practice.ipynb
import numpy as np
import pandas as pd

cities = get_nearest_city(-45.000, 45.000, 5)
# cities
city_data_df = pd.DataFrame(get_weather_by_city(cities))
city_data_df.to_csv("..\weather_database\cities_new.csv")
# print(city_url)

In [ ]:
city_data_df.head()

In [ ]:
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]

In [ ]:
import time
import matplotlib.pyplot as plt

# build the scatter plots for latitude versus max temperature
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(lats,
            max_temps,
            edgecolor="black",
            linewidths=1,
            marker="o",
            alpha=0.8,
            label="Cities")
# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")

plt.grid(True)

# Save the figure.
plt.savefig("weather_data/Fig1.png")
# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. humidity.
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(lats,
            humidity,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")

plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig2.png")
# Show plot.
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. cloudiness.
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(lats,
            cloudiness,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Cloudiness (%) "+ time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig3.png")
# Show plot.
plt.show()

### 6.3.4 Plot Latitude vs. Wind Speed


In [ ]:
# Build the scatter plots for latitude vs. wind speed.
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(lats,
            wind_speed,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Wind Speed "+ time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig4.png")
# Show plot.
plt.show()

In [ ]:
# import lineregress
from scipy.stats import linregress

# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):
    # Run regression on hemisphere weather data.
    (slope, intercept, r_value, p_value, std_err) = linregress(x_values, y_values)

    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x_values * slope + intercept
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Create a scatter plot and plot the regression line.
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")
    plt.xlabel('Latitude')
    plt.ylabel(y_label)
    plt.show()

In [ ]:
index_13 = city_data_df.loc[13]
index_13

In [ ]:
northern_hemi_df = city_data_df[city_data_df['Lat']>=0]
southern_hemi_df = city_data_df[city_data_df['Lat']<0]
# northern_hemi_df.head()
# southern_hemi_df.head()
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Humidity"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Maximum Temperature', 'Humidity',(-1,19))

In [ ]:
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Max Temp"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for Maximum Temperature', 'Max Temp',(10,40))

In [ ]:
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Humidity"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Northern Hemisphere \
                        for % Humidity', '% Humidity',(40,10))

In [ ]:
# Linear regression on the Southern Hemisphere
# southern_hemi_df = city_data_df[city_data_df['Lat']<0]
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Humidity"]
print(y_values, x_values)
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for % Humidity', '% Humidity',(-50,15))

In [ ]:
# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Cloudiness"]
# Call the function.
plot_linear_regression(x_values, y_values,
                       'Linear Regression on the Southern Hemisphere \
                        for % Cloudiness', '% Cloudiness',(-50,60))